In [ ]:
pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 19

In [ ]:
import pandas as pd
c_df = pd.read_csv("/content/drive/MyDrive/CMT/commentss.csv",encoding="utf-8")
c_df.head(3)

,INDEX,LABEL
0,買來用做PS和Pr等，性能非常強大，外觀非常好看，很輕很薄，界面顏色很好看，很流暢，視網膜屏...,0
1,按事情音質非常不錯！屏幕分辨率煥然一新！第一次入手蘋果生態！感覺好極了。鍵盤也很耐用愛了愛了...,0
2,性能非常強大，外觀非常好看，很輕很薄，界面顏色很好看，很流暢，可以做多用途，京東自營肯定有正...,0


In [ ]:
c_df.shape

(13999, 2)

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(c_df, test_size = 0.2, random_state = 9527)

In [ ]:
import torch
import urllib3, socket
from urllib3.connection import HTTPConnection
from simpletransformers.classification import ClassificationModel

HTTPConnection.default_socket_options = ( 
        HTTPConnection.default_socket_options + [
        (socket.SOL_SOCKET, socket.SO_SNDBUF, 1000000), #1MB in byte
        (socket.SOL_SOCKET, socket.SO_RCVBUF, 1000000)
    ])

model = ClassificationModel('bert', 'hfl/chinese-bert-wwm', use_cuda=torch.cuda.is_available())
model.train_model(train_df, args = {'overwrite_output_dir': True})

Some weights of the model checkpoint at hfl/chinese-bert-wwm were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/11199 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1400 [00:00<?, ?it/s]

(1400, 0.07851780389917362)

In [ ]:
t_df0 = c_df.head(2800)
t_df0.shape

(2800, 2)

In [ ]:
t_df1 = c_df[['INDEX','LABEL']][2801:10000]
t_df1.shape

(7199, 2)

In [ ]:
result, model_output, wrong_predictions = model.eval_model(t_df1)
result

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/7199 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/900 [00:00<?, ?it/s]

{'mcc': 0.9952235403084694,
 'tp': 3995,
 'tn': 3187,
 'fp': 2,
 'fn': 15,
 'auroc': 0.9999748199272906,
 'auprc': 0.9999802251987534,
 'eval_loss': 0.0126952140136402}

In [ ]:
tp = 1177
tn = 1607
fp = 9
fn = 7

In [ ]:
tp = 0
tn = 2782
fp = 18
fn = 0

In [ ]:
tp = 3997
tn = 3186
fp = 3
fn = 13

In [ ]:
tp = 5978
tn = 7967
fp = 23
fn = 31

In [ ]:
print("混亂矩陣結果為\n[[", tp, " ", fp, "]\n [", fn, " ", tn, "]]")

混亂矩陣結果為
[[ 3997   3 ]
 [ 13   3186 ]]


In [ ]:
#整體準確率 Accuracy
tol_acu = (tp + tn) / (tp + tn + fp + fn)
print("整體準確率:",tol_acu, "%", end="")

整體準確率: 0.9977774690929295 %

In [ ]:
#好評準確率 Precision
r_acu = tp / (tp + fp)
print("好評準確率:", r_acu, "%", end="")

好評準確率: 0.99925 %

In [ ]:
#Recall
rc = tp / (tp + fn)
rc

0.9967581047381546

In [ ]:
#壞評準確率
f_acu = tn / (fn + tn)
print("壞評準確率:", f_acu, "%", end="")

壞評準確率: 0.9959362300718975 %

In [ ]:
#F1
f1 = 2 * (r_acu*rc/(r_acu+rc))
f1

0.9980024968789014